## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-23-02-10-39 +0000,bbc,Ghislaine Maxwell claims in DOJ interview that...,https://www.bbc.com/news/articles/cdd3pe6189go...
1,2025-08-23-02-07-22 +0000,wapo,"Texas, Florida ask to join mifepristone case, ...",https://www.washingtonpost.com/nation/2025/08/...
2,2025-08-23-01-59-20 +0000,nyt,"Abrego Garcia, Free From Tennessee Jail, Is Re...",https://www.nytimes.com/2025/08/22/us/politics...
3,2025-08-23-01-57-11 +0000,latimes,A former Superior Court judge accused of two s...,https://www.latimes.com/california/story/2025-...
4,2025-08-23-01-54-40 +0000,nyt,Trump Administration Orders Work Halted on Win...,https://www.nytimes.com/2025/08/22/climate/tru...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2301/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
39,trump,58
82,new,23
388,tariffs,13
37,up,13
116,national,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
166,2025-08-22-18-08-00 +0000,wsj,The FBI searched the Maryland home and D.C. of...,https://www.wsj.com/politics/policy/fbi-raids-...,199
33,2025-08-22-23-43-00 +0000,wsj,Ghislaine Maxwell said she never saw President...,https://www.wsj.com/us-news/law/justice-depart...,137
205,2025-08-22-16-19-00 +0000,wsj,Defense Secretary Pete Hegseth has authorized ...,https://www.wsj.com/politics/policy/defense-se...,136
175,2025-08-22-17-46-20 +0000,nypost,How John Bolton’s controversial 2020 memoir sp...,https://nypost.com/2025/08/22/us-news/how-john...,124
181,2025-08-22-17-22-48 +0000,nyt,"Who Is John Bolton, Trump’s Former National Se...",https://www.nytimes.com/2025/08/22/us/politics...,114


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
166,199,2025-08-22-18-08-00 +0000,wsj,The FBI searched the Maryland home and D.C. of...,https://www.wsj.com/politics/policy/fbi-raids-...
33,79,2025-08-22-23-43-00 +0000,wsj,Ghislaine Maxwell said she never saw President...,https://www.wsj.com/us-news/law/justice-depart...
156,73,2025-08-22-18-33-00 +0000,wsj,Canada’s decision to drop tariffs on most U.S....,https://www.wsj.com/articles/canadas-rollback-...
328,57,2025-08-22-03-25-47 +0000,nypost,Federal judge orders some parts of ‘Alligator ...,https://nypost.com/2025/08/21/us-news/federal-...
317,54,2025-08-22-06-08-37 +0000,wsj,"Stock Market News, Aug. 22, 2025: Dow Gains as...",https://www.wsj.com/livecoverage/stock-market-...
272,50,2025-08-22-10-45-16 +0000,nypost,Israel warns Hamas that ‘gates of hell’ will o...,https://nypost.com/2025/08/22/world-news/israe...
266,49,2025-08-22-11-07-29 +0000,nypost,"Putin demands Ukraine surrender Donbas, give u...",https://nypost.com/2025/08/22/world-news/putin...
14,43,2025-08-23-01-00-00 +0000,wsj,Texas’ redistricting plan has set off a nation...,https://www.wsj.com/politics/a-partisan-shot-f...
258,41,2025-08-22-12-05-57 +0000,wapo,He agreed to conduct a band when he turned 100...,https://www.washingtonpost.com/lifestyle/2025/...
213,39,2025-08-22-15-42-14 +0000,nypost,Ex-Adams adviser Winnie Greco claims cash-stuf...,https://nypost.com/2025/08/22/us-news/mayor-er...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
